In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("white")

# plot colors
nice_colors = {
    "TES [GWh]": "orange",
    "Battery [GWh]": "darkgrey",
    "H2 storage [GWh]": "#9467bd",
}

# for technologies and plot legend
nice_names = {
    "HVDC overhead": "HVDC [GW]",
    "battery storage": "Battery [GWh]",
    "csp-tower TES": "TES [GWh]",
    "hydrogen storage tank type 1": "H2 storage [GWh]",
    "H2 (g) pipeline": "H2 Pipeline [GW]",
    "electrolysis": "H2 Electrolysis [GW]",
    "CCGT": "H2 gas turbine [GW]",
}

# magic number from efficiencies.csv; report TES storage and H2 storage in MWh electricity rather than heat
efficiency_csp_power_block = 0.41
efficiency_h2_ccgt = 0.58

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")

df = df.loc[(df["exporter"] == snakemake.wildcards["exporter"])
            & (df["category"] == "installed capacity")
            & (df["scenario"].str.contains(snakemake.wildcards["csp"]))
            & (df["esc"].isin(["hvdc-to-elec","pipeline-h2-to-elec"]))
           ]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "Baseload",
    #    "daily": "Daily",
    #    "weekly": "Weekly",
    #    "biweekly": "Biweekly",
    #    "monthly": "Month",
    #    "quaterly": "Quarterly",
    "annually": "Annual",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

# Restrict to flexibilities of interest
df = df.loc[df["flexibility"].isin(flexibilities.values())]

# Extract technology to aggreagate/group by later
# First replace removes numbers, e.g. csp-tower 12 -> csp-tower
# Second replace removes "(exp)" and "(imp)" brackets
# Order of replaces matters, e.g. "hydrogen storage tank type 1 (exp)" keeps the number because
# "(exp)" is removed after the number replacement
df["technology"] = df["subcategory"].str.replace("\s+\d+$","",regex=True).replace("\s\(.+\)$","", regex=True)

# Aggregate by technologies
df = df.groupby(["esc","flexibility","technology"])["value"].sum()

# Pivot to long columns format for plotting
df = df.to_frame().pivot_table(index=["esc","flexibility"], columns="technology",values="value")

# Select only wanted technologies
df = df.loc[:,df.columns.isin(nice_names.keys())]

# Rename for nicer names in plot/handling
df = df.rename(columns=nice_names)

# CSP-TES and H2 technologies in MWh_e/MW_e electricity equivalents
df.loc[:,df.columns.isin(["TES [GWh]"])] *= efficiency_csp_power_block
df.loc[:,df.columns.isin(["H2 storage [GWh]"])] *= efficiency_h2_ccgt
df.loc[:,df.columns.isin(["H2 Pipeline [GW]"])] *= efficiency_h2_ccgt
df.loc[:,df.columns.isin(["H2 gas turbine [GW]"])] *= efficiency_h2_ccgt

# Correct sort order (HVDC first, then pipeline; Yearly first, then no flex)
df = df.sort_index(ascending=[True, False])

# Unit conversion: in GWh and GW
df /= 1e3

# Do not plot 0-value data
df = df.replace({0:np.nan})

# Plotting
fig, ax = plt.subplots(figsize=(6, 3))

# Seperator between ESCs
ax.plot([1.5] * 2, [0, 250], ls="dashed", color="grey")

ax.text(0.5, 175 + 25 / 2, "HVDC", va="center", ha="center")
ax.text(2.5, 175 + 25 / 2, "H2 Pipeline", va="center", ha="center")

tmp = df.loc[:,df.columns.isin(["Battery [GWh]", "TES [GWh]", "H2 storage [GWh]"])]
tmp.plot.bar(ax=ax, width=0.2*len(tmp.columns), stacked=False, color=[nice_colors[tech] for tech in tmp.columns], position=.7)

# Right y-axis for lower value, GW capacities
axr = ax.twinx()
df["HVDC [GW]"].plot.line(ls="None", marker="*", color="black", ax=axr)
df["H2 Pipeline [GW]"].plot.line(ls="None", marker="o", color="black", markerfacecolor="white", ax=axr)
df["H2 Electrolysis [GW]"].plot.line(ls="None", marker="s", color="black", markerfacecolor="white", ax=axr)
df["H2 gas turbine [GW]"].plot.line(ls="None", marker="2", color="black", markerfacecolor="white", ax=axr)

ax.set_ylabel("Capacity [$\mathregular{GWh_{e}}$]")
axr.set_ylabel("Capacity [$\mathregular{GW_{e}}$]")
ax.set_xlabel("")
ax.set_xticklabels(["Annual", "Baseload", "Annual", "Baseload"], rotation=0, fontsize="small")

# Add subscript $_e$ to all units as indicator for electricity equivalents
new_labels = ax.get_legend_handles_labels()[1][::-1] + axr.get_legend_handles_labels()[1]
new_labels = [l.split("[")[0]+"[$\\mathregular{"+l.split("[")[1][:-1]+"_e}$]" for l in new_labels]
# Combine add legend handles/labels into one legend
ax.legend(
    handles=ax.get_legend_handles_labels()[0][::-1] + axr.get_legend_handles_labels()[0],
    labels=new_labels,
    fontsize="small",
    columnspacing=1.0,
    bbox_to_anchor=(1.2, 0),
    loc="lower left",
)

axr.legend().remove()

# Add smaller, secondary y-axis grid without labels
ax.set_axisbelow(True)
ax.set_yticks(np.arange(25, 275, 50), minor=True)
axr.set_yticks(np.arange(2.5, 27.5, 5), minor=True)
ax.yaxis.grid(which="major")
ax.yaxis.grid(which="minor", ls="--")


# align major gridlines between left and right y axis
ax.set_yticks(np.arange(0,250,50))
axr.set_yticks(np.arange(0,25,5))
ax.set_xlim(-0.5, 3.5)
ax.set_ylim(0, 200 * 1.05) # Magic values for consistency for all ESCs
axr.set_ylim(0, 20 * 1.05)

fig.tight_layout()

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")